In [1]:
import os 
import betfairlightweight
from betfairlightweight.resources import MarketBook, RunnerBook
from betfairlightweight import filters
import pandas as pd
import numpy as np
import os
import datetime
import json
from matplotlib import pyplot as plt
from typing import Tuple, List

In [3]:
#CERTS_PATH = "/Users/charaka/Desktop/University/Msc Machine Learning & Data Science/Masters Project/certs/"
APP_KEY = "DY9dGbHPottkhJsl"

USERNAME = "bex_mclean19@icloud.com"
PASSWORD = "Nath1908!"

In [4]:
#pip install betfairlightweight
#%pip install pandas
#%pip install numpy matplotlib

In [5]:
def deltaTime(seconds=0, minutes=0, days=0, weeks=0):
    return (
        datetime.datetime.utcnow() + datetime.timedelta(seconds=seconds, minutes=minutes, days=days, weeks=weeks)
    ).strftime("%Y-%m-%dT%TZ")


### Login to Betfair API

In [6]:
trading = betfairlightweight.APIClient(username=USERNAME, password=PASSWORD, app_key=APP_KEY, certs=CERTS_PATH)
trading.login()

NameError: name 'CERTS_PATH' is not defined

### Get the list of markets

In [8]:
event_types = trading.betting.list_event_types()

sport_ids = pd.DataFrame({
    'Sport': [event_type_object.event_type.name for event_type_object in event_types],
    'ID': [event_type_object.event_type.id for event_type_object in event_types]
}).set_index('Sport').sort_index()

sport_ids

,ID
Sport,
American Football,6423
Australian Rules,61420
Baseball,7511
Basketball,7522
Boxing,6
Cricket,4
Cycling,11
Darts,3503
Esports,27454571


### Filter for the market we want e.g: Horse Racing

In [6]:
# Filter for just horse racing
horse_racing_filter = betfairlightweight.filters.market_filter(text_query='Horse Racing')

# This returns a list
horse_racing_event_type = trading.betting.list_event_types(
    filter=horse_racing_filter)

# Get the first element of the list
horse_racing_event_type = horse_racing_event_type[0]

horse_racing_event_type_id = horse_racing_event_type.event_type.id
print(f"Event Type ID [Horse Racing]: {horse_racing_event_type_id}")

Event Type ID [Horse Racing]: 7


### Get next week Football competitions

In [10]:
# Get a datetime object in a week and convert to string
datetime_in_a_week = deltaTime(weeks=1)

# Create a competition filter
competition_filter = betfairlightweight.filters.market_filter(
    event_type_ids=[1], # Soccer's event type id is 1
    market_start_time={
        'to': datetime_in_a_week
    })

# Get a list of competitions for soccer
competitions = trading.betting.list_competitions(
    filter=competition_filter
)

# Iterate over the competitions and create a dataframe of competitions and competition ids
soccer_competitions = pd.DataFrame({
    'Competition': [competition_object.competition.name for competition_object in competitions],
    'ID': [competition_object.competition.id for competition_object in competitions]
})

soccer_competitions

,Competition,ID
0,Specials,2608550
1,Brazilian Carioca Matches,12164283
2,CONMEBOL U20 Championship,12185403
3,German Regionalliga Northeast,318343
4,English Northern Premier League,12200889
...,...,...
81,Belgian First Division A,89979
82,Swiss Cup,29051
83,Czech Cup,12227137
84,Hungarian Cup,11503700


### View all available markets till the next day (football)

In [4]:
# Define a market filter
football_event_filter = betfairlightweight.filters.market_filter(
    event_type_ids=[1],
    market_countries=['GB'],
    market_start_time={
        'to': deltaTime(days=1)
    }
)

gb_football_events = trading.betting.list_events(
    filter=football_event_filter
)

football_events_today = pd.DataFrame({
    'Event Name': [event_object.event.name for event_object in gb_football_events],
    'Event ID': [event_object.event.id for event_object in gb_football_events],
    'Event Venue': [event_object.event.venue for event_object in gb_football_events],
    'Country Code': [event_object.event.country_code for event_object in gb_football_events],
    'Time Zone': [event_object.event.time_zone for event_object in gb_football_events],
    'Open Date': [event_object.event.open_date for event_object in gb_football_events],
    'Market Count': [event_object.market_count for event_object in gb_football_events]
})

football_events_today

,Event Name,Event ID,Event Venue,Country Code,Time Zone,Open Date,Market Count
0,Frome Town v Bishops Cleeve,32076393,None,GB,GMT,2023-02-03 19:45:00,24
1,Queens Park v Ayr,32063497,None,GB,GMT,2023-02-03 19:45:00,24
2,Chelsea v Fulham,32031883,None,GB,GMT,2023-02-03 20:00:00,62
3,English League 2,1908056,None,GB,Europe/London,2019-08-05 14:00:00,4
4,Colchester United U21 v Cardiff City U21,32075022,None,GB,GMT,2023-02-03 12:01:04,22
5,English Championship,1908053,None,GB,Europe/London,2021-08-06 18:45:00,7
6,Daily Goals,32071392,None,GB,GMT,2023-02-04 12:30:00,1
7,Bangor v Crusaders,32076419,None,GB,GMT,2023-02-03 19:45:00,24
8,English League 1,1908054,None,GB,Europe/London,2020-09-12 14:00:00,4
9,Everton v Arsenal,32031876,None,GB,GMT,2023-02-04 12:30:00,62


### View market types

In [5]:
event_id = football_events_today['Event ID'][0]

market_types_filter = betfairlightweight.filters.market_filter(event_ids=[event_id])

# Request market types
market_types = trading.betting.list_market_types(
    filter=market_types_filter
)

# Create a DataFrame of market types
market_types_football= pd.DataFrame({
    'Market Type': [market_type_object.market_type for market_type_object in market_types],
})

market_types_football.head()

,Market Type
0,CORRECT_SCORE
1,OVER_UNDER_55
2,ALT_TOTAL_GOALS
3,BOTH_TEAMS_TO_SCORE
4,DOUBLE_CHANCE


### Get Market Catalogues

In [7]:
market_catalogue_filter = betfairlightweight.filters.market_filter(event_ids=[event_id], market_type_codes=['MATCH_ODDS'])

market_catalogues = trading.betting.list_market_catalogue(
    filter=market_catalogue_filter,
    max_results='100',
    sort='FIRST_TO_START'
)

# Create a DataFrame for each market catalogue
market_types_football = pd.DataFrame({
    'Market Name': [market_cat_object.market_name for market_cat_object in market_catalogues],
    'Market ID': [market_cat_object.market_id for market_cat_object in market_catalogues],
    'Total Matched': [market_cat_object.total_matched for market_cat_object in market_catalogues],
})

market_types_football

,Market Name,Market ID,Total Matched
0,Match Odds,1.209500605,2.0


In [8]:
market_catalogue_filter

{'eventIds': ['32076393'], 'marketTypeCodes': ['MATCH_ODDS']}

In [33]:
event_ids = ["32076393", "32031883"]

event_filter = betfairlightweight.filters.market_filter(
    event_type_ids=[1],
    market_countries=['GB'],
    event_ids=event_ids,
    market_start_time={
        'to': deltaTime(days=1)
    }
)

all_events = trading.betting.list_events(
    filter=event_filter,
    lightweight=True
)

events_obj = []

for event in all_events:
    event= event["event"]
    
    catalogue = trading.betting.list_market_catalogue(
        filter=betfairlightweight.filters.market_filter(event_ids=[event["id"]], market_type_codes=['MATCH_ODDS']),
        max_results='100',
        sort='FIRST_TO_START',
        lightweight=True
    )

    market_ids = [market["marketId"] for market in catalogue]

    if len(market_ids) == 0:
        continue

    markets = trading.betting.list_market_book(
        market_ids=market_ids,
        lightweight=True
    )

    events_obj.append({ "event": event, "markets": markets })

32076393 Event Name: Frome Town v Bishops Cleeve
Market IDs: ['1.209500605']
Markets: [{'marketId': '1.209500605', 'isMarketDataDelayed': False, 'status': 'OPEN', 'betDelay': 0, 'bspReconciled': False, 'complete': True, 'inplay': False, 'numberOfWinners': 1, 'numberOfRunners': 3, 'numberOfActiveRunners': 3, 'totalMatched': 0.0, 'totalAvailable': 832.94, 'crossMatching': True, 'runnersVoidable': False, 'version': 5026809415, 'runners': [{'selectionId': 5959623, 'handicap': 0.0, 'status': 'ACTIVE', 'totalMatched': 0.0}, {'selectionId': 1282853, 'handicap': 0.0, 'status': 'ACTIVE', 'totalMatched': 0.0}, {'selectionId': 58805, 'handicap': 0.0, 'status': 'ACTIVE', 'totalMatched': 0.0}]}]
32031883 Event Name: Chelsea v Fulham
Market IDs: ['1.208729473']
Markets: [{'marketId': '1.208729473', 'isMarketDataDelayed': False, 'status': 'OPEN', 'betDelay': 0, 'bspReconciled': False, 'complete': True, 'inplay': False, 'numberOfWinners': 1, 'numberOfRunners': 3, 'numberOfActiveRunners': 3, 'lastMatch

In [28]:
catalogue = trading.betting.list_market_catalogue(
        filter=betfairlightweight.filters.market_filter(event_ids=[event_id]),
        max_results='100',
        sort='FIRST_TO_START',
        lightweight=True
    )

In [29]:
catalogue

[{'marketId': '1.209500614',
  'marketName': 'Correct Score',
  'totalMatched': 0.0},
 {'marketId': '1.209500613',
  'marketName': 'Over/Under 5.5 Goals',
  'totalMatched': 0.0},
 {'marketId': '1.209500615', 'marketName': 'Goal Lines', 'totalMatched': 0.0},
 {'marketId': '1.209500689',
  'marketName': 'Both teams to Score?',
  'totalMatched': 0.0},
 {'marketId': '1.209500606',
  'marketName': 'Double Chance',
  'totalMatched': 0.0},
 {'marketId': '1.209500605', 'marketName': 'Match Odds', 'totalMatched': 0.0},
 {'marketId': '1.209500608',
  'marketName': 'First Half Goals 0.5',
  'totalMatched': 0.0},
 {'marketId': '1.209500607',
  'marketName': 'Frome Town +1',
  'totalMatched': 0.0},
 {'marketId': '1.209500610',
  'marketName': 'First Half Goals 1.5',
  'totalMatched': 0.0},
 {'marketId': '1.209500609',
  'marketName': 'Over/Under 0.5 Goals',
  'totalMatched': 0.0},
 {'marketId': '1.209500612', 'marketName': 'Half Time', 'totalMatched': 0.0},
 {'marketId': '1.209500611',
  'marketNam

In [19]:
events

[{'event': {'id': '32076393',
   'name': 'Frome Town v Bishops Cleeve',
   'countryCode': 'GB',
   'timezone': 'GMT',
   'openDate': '2023-02-03T19:45:00.000Z'},
  'marketCount': 24},
 {'event': {'id': '32063497',
   'name': 'Queens Park v Ayr',
   'countryCode': 'GB',
   'timezone': 'GMT',
   'openDate': '2023-02-03T19:45:00.000Z'},
  'marketCount': 24},
 {'event': {'id': '32031883',
   'name': 'Chelsea v Fulham',
   'countryCode': 'GB',
   'timezone': 'GMT',
   'openDate': '2023-02-03T20:00:00.000Z'},
  'marketCount': 62},
 {'event': {'id': '1908056',
   'name': 'English League 2',
   'countryCode': 'GB',
   'timezone': 'Europe/London',
   'openDate': '2019-08-05T14:00:00.000Z'},
  'marketCount': 4},
 {'event': {'id': '32075022',
   'name': 'Colchester United U21 v Cardiff City U21',
   'countryCode': 'GB',
   'timezone': 'GMT',
   'openDate': '2023-02-03T12:00:00.000Z'},
  'marketCount': 24},
 {'event': {'id': '1908053',
   'name': 'English Championship',
   'countryCode': 'GB',
   

In [21]:
markets

[{'marketId': '1.209500613',
  'isMarketDataDelayed': False,
  'status': 'OPEN',
  'betDelay': 0,
  'bspReconciled': False,
  'complete': True,
  'inplay': False,
  'numberOfWinners': 1,
  'numberOfRunners': 2,
  'numberOfActiveRunners': 2,
  'totalMatched': 0.0,
  'totalAvailable': 184.86,
  'crossMatching': True,
  'runnersVoidable': False,
  'version': 5026809425,
  'runners': [{'selectionId': 1485567,
    'handicap': 0.0,
    'status': 'ACTIVE',
    'totalMatched': 0.0},
   {'selectionId': 1485568,
    'handicap': 0.0,
    'status': 'ACTIVE',
    'totalMatched': 0.0}]},
 {'marketId': '1.209500614',
  'isMarketDataDelayed': False,
  'status': 'OPEN',
  'betDelay': 0,
  'bspReconciled': False,
  'complete': True,
  'inplay': False,
  'numberOfWinners': 1,
  'numberOfRunners': 19,
  'numberOfActiveRunners': 19,
  'totalMatched': 0.0,
  'totalAvailable': 204.15,
  'crossMatching': True,
  'runnersVoidable': False,
  'version': 5026809421,
  'runners': [{'selectionId': 1,
    'handicap'

In [15]:
def process_runner_books(runner_books):
    '''
    This function processes the runner books and returns a DataFrame with the best back/lay prices + vol for each runner
    :param runner_books:
    :return:
    '''
    best_back_prices = [runner_book.ex.available_to_back[0].price
        if runner_book.ex.available_to_back[0].price
        else 1.01
        for runner_book
        in runner_books]
    best_back_sizes = [runner_book.ex.available_to_back[0].size
        if runner_book.ex.available_to_back[0].size
        else 1.01
        for runner_book
        in runner_books]

    best_lay_prices = [runner_book.ex.available_to_lay[0].price
        if runner_book.ex.available_to_lay[0].price
        else 1000.0
        for runner_book
        in runner_books]
    best_lay_sizes = [runner_book.ex.available_to_lay[0].size
        if runner_book.ex.available_to_lay[0].size
        else 1.01
        for runner_book
        in runner_books]

    selection_ids = [runner_book.selection_id for runner_book in runner_books]
    last_prices_traded = [runner_book.last_price_traded for runner_book in runner_books]
    total_matched = [runner_book.total_matched for runner_book in runner_books]
    statuses = [runner_book.status for runner_book in runner_books]
    scratching_datetimes = [runner_book.removal_date for runner_book in runner_books]
    adjustment_factors = [runner_book.adjustment_factor for runner_book in runner_books]

    df = pd.DataFrame({
        'Selection ID': selection_ids,
        'Best Back Price': best_back_prices,
        'Best Back Size': best_back_sizes,
        'Best Lay Price': best_lay_prices,
        'Best Lay Size': best_lay_sizes,
        'Last Price Traded': last_prices_traded,
        'Total Matched': total_matched,
        'Status': statuses,
        'Removal Date': scratching_datetimes,
        'Adjustment Factor': adjustment_factors
    })
    return df


### Build a Snaphot of The Current Order Book

In [56]:
class RunnerSnapshot:
    def __init__(self, cur_runner_book: RunnerBook) -> None:
        self.id = cur_runner_book.selection_id
        self.atl_book = [[atl.price, atl.size] for atl in cur_runner_book.ex.available_to_lay] # Available to Lay
        self.atb_book = [[atl.price, atl.size] for atl in cur_runner_book.ex.available_to_back] # Available to Buy
        self.ltp = cur_runner_book.last_price_traded # Last Traded Price
        self.tv = cur_runner_book.total_matched # Total Volume
        self.status = cur_runner_book.status

    def _get_book_price_volume(self, book, type=None, limit=0) -> Tuple[List[float], List[float]]:
        data = sorted(book, key=lambda x: x[0])

        if limit > 0:
            if type == 'back':
                data = data[-limit:]
            elif type == 'lay':
                data = data[:limit]
            else:
                raise Exception("Invalid type")

        prices = []
        volumes = []
        for price, volume in data:
            prices.append(str(price))
            volumes.append(volume)
        return prices, volumes

    def view(self, limit=0):
        """ View the current stauts of the order book for a runner 
        
        Args:
            limit (int, optional): Limit the number of prices to display. Defaults to all.
        """
        backs = self.atb_book
        lays = self.atl_book
            
        backs_price, backs_volume = self._get_book_price_volume(backs, 'back', limit)
        lays_price, lays_volume = self._get_book_price_volume(lays, 'lay', limit)
        plt.figure(figsize=(10, 2))
        plt.bar(backs_price, backs_volume, color='green')
        plt.axvline(x=str(self.ltp), color='k', linewidth=1.0, linestyle='--', label='Last Traded Price') 
        plt.bar(lays_price, lays_volume, color='red')
        plt.legend()
        plt.show()

class OrderBookSnapshot:
    def __init__(self, cur_market_book: MarketBook) -> None:
        self.market_book = cur_market_book.runners
        self.runners = {runner.selection_id: RunnerSnapshot(runner) for runner in cur_market_book.runners}
        self.total_matched = sum(runner.tv for runner in self.runners.values())

    def view(self, runnerId: str = None, limit: int=0) -> None:

        if runnerId is None:
            for _, runner in self.runners.items():
                runner.view(limit)
            return

        if runnerId not in self.runners.keys():
            return

        self.runners[runnerId].view(limit)

In [46]:
market_id = "1.209348802"
# Create a price filter. Get all traded and offer data
price_filter = betfairlightweight.filters.price_projection(
    price_data=['EX_BEST_OFFERS']
)

# Request market books
market_books = trading.betting.list_market_book(
    market_ids=[market_id],
    price_projection=price_filter
)

# Grab the first market book from the returned list as we only requested one market 
market_book = market_books[0]

OrderBookSnapshot(market_book).view()

,Selection ID,Best Back Price,Best Back Size,Best Lay Price,Best Lay Size,Last Price Traded,Total Matched,Status,Removal Date,Adjustment Factor
0,7840178,5.00,64.25,5.50,33.0,4.90,3543.66,ACTIVE,None,None
1,6692741,3.05,79.00,3.25,69.0,3.00,3959.98,ACTIVE,None,None
2,58805,1.99,52.00,2.08,96.0,2.34,633.83,ACTIVE,None,None


### Logout of Betfair API

In [20]:
trading.logout()

<LogoutResource>

In [66]:
pd.DataFrame({ 'ID': [1, 2, 3], 'Name': ['A', 'B', 'C'] })


,ID,Name
0,1,A
1,2,B
2,3,C
